**YOLO Training Notebook** 



In [11]:
#Config 
from pathlib import Path

DATA_YAML = "/Users/shatha/Desktop/privacy-blurring-system/final_data/dataset.yaml"

# Training setup
WEIGHTS   = "yolov8s-obb.pt"   
EPOCHS    = 80                 
IMG_SIZE  = 640                
BATCH     = 8                
SEED      = 42

# Project/run naming
PROJECT   = "models"

# Predict setup
CONF      = 0.25
IOU       = 0.5
TEST_DIR  = "/Users/shatha/Desktop/privacy-blurring-system/test_images"  # put a few new images here

# Device: "cpu" for Mac; "0" for first CUDA GPU
DEVICE    = "cpu"              # change to "0" on Colab GPU
print("CONFIG OK")


CONFIG OK


In [14]:
#Train (OBB)
model = YOLO(WEIGHTS)  # OBB weights

train_args = dict(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    device=DEVICE,
    project=PROJECT,
    name=RUNNAME,
    seed=SEED,
    pretrained=True,
    optimizer="SGD",          
    lr0=0.01, lrf=0.01,       
    momentum=0.937,
    weight_decay=0.0005,
    cos_lr=True,
    patience=25,              # early stop if no val improvement
    close_mosaic=10,         
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    degrees=5.0, translate=0.05, scale=0.1, shear=3.0, flipud=0.0, fliplr=0.0,  
    box=7.5, cls=0.5, dfl=1.5 
)

res = model.train(**train_args)


New https://pypi.org/project/ultralytics/8.3.195 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.194 🚀 Python-3.12.7 torch-2.8.0 CPU (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/Users/shatha/Desktop/privacy-blurring-system/final_data/dataset.yaml, degrees=5.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-obb.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=obb_fixed_v15, nbs=64, nms=Fa

/Users/shatha/Desktop/privacy-blurring-system/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 1.6±1.2 ms, read: 456.1±98.2 MB/s, size: 1059.0 KB)
val: Scanning /Users/shatha/Desktop/privacy-blurring-system/final_data/val/labels.cache... 200 images, 0 backgrounds, 94 corrupt: 100% ━━━━━━━━━━━━ 200/200 501.1Kit/s 0.0s
val: /Users/shatha/Desktop/privacy-blurring-system/final_data/val/images/alb_id_01.jpg: ignoring corrupt image/label: labels require 5 columns, 6 columns detected
val: /Users/shatha/Desktop/privacy-blurring-system/final_data/val/images/alb_id_13.jpg: ignoring corrupt image/label: labels require 5 columns, 6 columns detected
val: /Users/shatha/Desktop/privacy-blurring-system/final_data/val/images/alb_id_18.jpg: ignoring corrupt image/label: labels require 5 columns, 6 columns detected
val: /Users/shatha/Desktop/privacy-blurri

       1/80         0G     0.9004      1.758      1.746          1        640: 100% ━━━━━━━━━━━━ 99/99 0.1it/s 12:561.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 14/14 0.7it/s 19.2s1.3s
                   all        106        106       0.99      0.979      0.994      0.939

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/80         0G     0.6166      0.701       1.49         19        640: 23% ━━╸───────── 23/99 0.2it/s 2:27<7:36

**Train on new data**

In [14]:
# 0) Imports and paths
from pathlib import Path
import shutil, random

ROOT = Path("/Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot")
IMG_DIR = ROOT / "images"
LBL_DIR = ROOT / "labels"

print("IMG_DIR:", IMG_DIR.exists(), IMG_DIR)
print("LBL_DIR:", LBL_DIR.exists(), LBL_DIR)


IMG_DIR: True /Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot/images
LBL_DIR: True /Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot/labels


In [15]:
#Pair images to labels robustly (handles _E / rot_E_) and split; rename labels to match image stems 
from pathlib import Path
import shutil, random, re

ROOT = Path("/Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot")
IMG_DIR = ROOT / "images"
LBL_DIR = ROOT / "labels"
assert IMG_DIR.exists() and LBL_DIR.exists(), f"Missing: {IMG_DIR} or {LBL_DIR}"

SPLIT = (0.8, 0.1, 0.1)
random.seed(42)

#Normalize function used for BOTH images and labels to match stems
def norm_stem(s: str) -> str:
    s = s.lower()
    s = s.replace("-", "_").replace(" ", "_")
    # remove any standalone or inline _E token (e.g., _E_, _E00, _E)
    s = re.sub(r"_e(?=(_|\d|$))", "", s)
    # also handle explicit rot_E_ -> rot_
    s = s.replace("rot_e_", "rot_")
    # collapse repeated underscores
    s = re.sub(r"__+", "_", s)
    # strip leading/trailing underscores
    s = s.strip("_")
    return s

#Index label files by normalized stem
label_index = {}
for lbl in LBL_DIR.glob("*.txt"):
    key = norm_stem(lbl.stem)
    label_index[key] = lbl

#Collect images and match to labels using normalized keys
img_exts = {".png", ".jpg", ".jpeg", ".bmp", ".webp"}
images = [p for p in IMG_DIR.iterdir() if p.is_file() and p.suffix.lower() in img_exts]

pairs, no_label = [], []
for img in images:
    key = norm_stem(img.stem)
    lbl = label_index.get(key)
    if lbl is not None and lbl.exists():
        pairs.append((img, lbl))
    else:
        no_label.append(img.name)

print(f"Pairs matched: {len(pairs)}")
print(f"No-label images: {len(no_label)}")
print("Examples (up to 15):", no_label[:15])

if not pairs:
    raise SystemExit("No pairs matched. Inspect naming or label folder.")

#Prepare/clean split dirs
for split in ["train","val","test"]:
    (ROOT/split/"images").mkdir(parents=True, exist_ok=True)
    (ROOT/split/"labels").mkdir(parents=True, exist_ok=True)
    # optional: clear old files to avoid duplicates
    for p in (ROOT/split/"images").glob("*"): 
        if p.is_file(): p.unlink()
    for p in (ROOT/split/"labels").glob("*"): 
        if p.is_file(): p.unlink()

#Split and copy; IMPORTANT: rename label to exactly match the IMAGE stem
random.shuffle(pairs)
n = len(pairs)
n_train = int(SPLIT[0]*n)
n_val   = int(SPLIT[1]*n)
splits = {
    "train": pairs[:n_train],
    "val":   pairs[n_train:n_train+n_val],
    "test":  pairs[n_train+n_val:]
}

for split, items in splits.items():
    dst_img_dir = ROOT/split/"images"
    dst_lbl_dir = ROOT/split/"labels"
    for img, lbl in items:
        shutil.copy(img, dst_img_dir/img.name)
        # rename label to match the image stem (includes _E if present)
        shutil.copy(lbl, dst_lbl_dir/f"{img.stem}.txt")

print(f"Done. train={len(splits['train'])}, val={len(splits['val'])}, test={len(splits['test'])}")


Pairs matched: 91
No-label images: 0
Examples (up to 15): []
Done. train=72, val=9, test=10


In [16]:
from pathlib import Path
def count_files(p, exts): 
    return sum(1 for x in p.iterdir() if x.is_file() and x.suffix.lower() in exts)

ROOT = Path("/Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot")
for split in ["train","val","test"]:
    ni = count_files(ROOT/f"{split}/images", {".png",".jpg",".jpeg",".bmp",".webp"})
    nl = count_files(ROOT/f"{split}/labels", {".txt"})
    print(f"[CHECK] {split:>5}: images={ni}, labels={nl}")


[CHECK] train: images=72, labels=72
[CHECK]   val: images=9, labels=9
[CHECK]  test: images=10, labels=10


In [5]:
# 1) Paths (EDIT ONLY IF NEEDED)
ROOT = Path("/Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot")
PREV_WEIGHTS = Path("/Users/shatha/Desktop/privacy-blurring-system/models/obb_fixed_v14/weights/best.pt")
PROJECT_DIR = Path("/Users/shatha/Desktop/privacy-blurring-system/models")
RUN_NAME = "BG_v14"
DATA_YAML = "/Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot/dataset_rot.yaml" 

assert (ROOT/"train/images").exists() and (ROOT/"train/labels").exists(), "train split not found"
assert (ROOT/"val/images").exists() and (ROOT/"val/labels").exists(), "val split not found"
assert (ROOT/"test/images").exists() and (ROOT/"test/labels").exists(), "test split not found"
assert PREV_WEIGHTS.exists(), f"weights not found: {PREV_WEIGHTS}"
print("OK: dataset and weights exist.")


OK: dataset and weights exist.


In [6]:
# 2) Load model (fixed previous weights)
model = YOLO(str(PREV_WEIGHTS))
model.info()


YOLOv8s-obb summary: 144 layers, 11,422,940 parameters, 0 gradients, 29.6 GFLOPs


(144, 11422940, 0, 29.636684799999998)

In [7]:
# 3) Train
train_res = model.train(
    data=str(DATA_YAML),
    epochs=30,
    batch=8,
    imgsz=960,
    device="cpu",    
    workers=0,        
    amp=False,       
    patience=20,
    close_mosaic=10,
    seed=42,
    project=str(PROJECT_DIR),
    name=RUN_NAME,
    exist_ok=True,
    task="obb",
)
print("Train dir:", train_res.save_dir)


Ultralytics 8.3.196 🚀 Python-3.12.7 torch-2.8.0 CPU (Apple M2)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot/dataset_rot.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/Users/shatha/Desktop/privacy-blurring-system/models/obb_fixed_v14/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=BG_v14, nbs=64, nms=False

/Users/shatha/Desktop/privacy-blurring-system/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.2±0.3 ms, read: 1958.8±467.6 MB/s, size: 7304.6 KB)
val: Scanning /Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot/val/labels.cache... 9 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 9/9 18.0Kit/s 0.0s
Plotting labels to /Users/shatha/Desktop/privacy-blurring-system/models/BG_v14/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
Image sizes 960 train, 960 val
Using 0 dataloader workers
Logging results to /Users/shatha/Desktop/privacy-blurring-system/models/BG_v14
Starti

In [8]:
# 4) Validate (val + test)

val_res = model.val(
    data=str(DATA_YAML),
    split="val",
    project=str(PROJECT_DIR),
    name=f"{RUN_NAME}_val",
    exist_ok=True,
    save=False,
)
print("VAL metrics:", val_res.results_dict)

test_res = model.val(
    data=str(DATA_YAML),
    split="test",
    project=str(PROJECT_DIR),
    name=f"{RUN_NAME}_test",
    exist_ok=True,
    save=False,
)
print("TEST metrics:", test_res.results_dict)


Ultralytics 8.3.196 🚀 Python-3.12.7 torch-2.8.0 CPU (Apple M2)
YOLOv8s-obb summary (fused): 81 layers, 11,412,732 parameters, 0 gradients, 29.4 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.8 ms, read: 2476.9±403.4 MB/s, size: 9276.0 KB)
val: Scanning /Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot/val/labels.cache... 9 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 9/9 10.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.5it/s 4.4s12.5s
                   all          9        379       0.94      0.924      0.945      0.863
                  card          9          9          1          1      0.995      0.979
                  face          9          9          1          1      0.995       0.97
                  text          9        361      0.819      0.773      0.846       0.64
Speed: 2.1ms preprocess, 319.9ms inference, 0.0ms loss, 5.6ms postprocess per image
Results sav

Python(23086) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.5it/s 4.0s10.7s
                   all         10        503      0.912      0.885      0.924      0.812
                  card         10         10      0.949          1      0.995       0.96
                  face         10         10      0.949          1      0.995      0.956
                  text         10        483      0.839      0.656      0.783       0.52
Speed: 2.3ms preprocess, 289.2ms inference, 0.0ms loss, 4.5ms postprocess per image
Results saved to /Users/shatha/Desktop/privacy-blurring-system/models/BG_v14_test
TEST metrics: {'metrics/precision(B)': 0.9120103653317165, 'metrics/recall(B)': 0.8852917308358256, 'metrics/mAP50(B)': 0.9244159923294505, 'metrics/mAP50-95(B)': 0.8119617653005599, 'fitness': 0.8232071880034489}


In [3]:
# 2) Evaluate on test split

assert DATA_YAML.exists(), f"Missing dataset yaml: {DATA_YAML}"

test_res = model.val(
    data=str(DATA_YAML),
    split="test",
    project=str(PROJECT_DIR),
    name="rot_test_eval",
    exist_ok=True,
    save=False,      
    device="cpu",   
    task="obb",
)
print("[TEST] metrics:", test_res.results_dict)


Ultralytics 8.3.196 🚀 Python-3.12.7 torch-2.8.0 CPU (Apple M2)
YOLOv8s-obb summary (fused): 81 layers, 11,412,732 parameters, 0 gradients, 29.4 GFLOPs
val: Fast image access ✅ (ping: 0.7±0.8 ms, read: 928.0±211.4 MB/s, size: 4466.3 KB)
val: Scanning /Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot/test/labels.cache... 10 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 10/10 670.2it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 0.2it/s 4.7s
                   all         10        503      0.912      0.885      0.924      0.812
                  card         10         10      0.949          1      0.995       0.96
                  face         10         10      0.949          1      0.995      0.956
                  text         10        483      0.839      0.656      0.783       0.52
Speed: 1.7ms preprocess, 354.5ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved

**New training**

In [3]:
#Pair images to labels robustly (handles _E / rot_E_) and split; rename labels to match image stems
from pathlib import Path
import shutil, random, re

ROOT = Path("/Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/hand")
IMG_DIR = ROOT / "img"
LBL_DIR = ROOT / "label"
assert IMG_DIR.exists() and LBL_DIR.exists(), f"Missing: {IMG_DIR} or {LBL_DIR}"

SPLIT = (0.8, 0.1, 0.1)
random.seed(42)

#Normalize function used for BOTH images and labels to match stems
def norm_stem(s: str) -> str:
    s = s.lower()
    s = s.replace("-", "_").replace(" ", "_")
    # remove any standalone or inline _E token (e.g., _E_, _E00, _E)
    s = re.sub(r"_e(?=(_|\d|$))", "", s)
    # also handle explicit rot_E_ -> rot_
    s = s.replace("rot_e_", "rot_")
    # collapse repeated underscores
    s = re.sub(r"__+", "_", s)
    # strip leading/trailing underscores
    s = s.strip("_")
    return s

#Index label files by normalized stem
label_index = {}
for lbl in LBL_DIR.glob("*.txt"):
    key = norm_stem(lbl.stem)
    label_index[key] = lbl

#Collect images and match to labels using normalized keys
img_exts = {".png", ".jpg", ".jpeg", ".bmp", ".webp"}
images = [p for p in IMG_DIR.iterdir() if p.is_file() and p.suffix.lower() in img_exts]

pairs, no_label = [], []
for img in images:
    key = norm_stem(img.stem)
    lbl = label_index.get(key)
    if lbl is not None and lbl.exists():
        pairs.append((img, lbl))
    else:
        no_label.append(img.name)

print(f"Pairs matched: {len(pairs)}")
print(f"No-label images: {len(no_label)}")
print("Examples (up to 15):", no_label[:15])

if not pairs:
    raise SystemExit("No pairs matched. Inspect naming or label folder.")

#Prepare/clean split dirs
for split in ["train","val","test"]:
    (ROOT/split/"images").mkdir(parents=True, exist_ok=True)
    (ROOT/split/"labels").mkdir(parents=True, exist_ok=True)
    # optional: clear old files to avoid duplicates
    for p in (ROOT/split/"images").glob("*"): 
        if p.is_file(): p.unlink()
    for p in (ROOT/split/"labels").glob("*"): 
        if p.is_file(): p.unlink()

#Split and copy; IMPORTANT: rename label to exactly match the IMAGE stem
random.shuffle(pairs)
n = len(pairs)
n_train = int(SPLIT[0]*n)
n_val   = int(SPLIT[1]*n)
splits = {
    "train": pairs[:n_train],
    "val":   pairs[n_train:n_train+n_val],
    "test":  pairs[n_train+n_val:]
}

for split, items in splits.items():
    dst_img_dir = ROOT/split/"images"
    dst_lbl_dir = ROOT/split/"labels"
    for img, lbl in items:
        shutil.copy(img, dst_img_dir/img.name)
        # rename label to match the image stem (includes _E if present)
        shutil.copy(lbl, dst_lbl_dir/f"{img.stem}.txt")

print(f"Done. train={len(splits['train'])}, val={len(splits['val'])}, test={len(splits['test'])}")


Pairs matched: 10
No-label images: 0
Examples (up to 15): []
Done. train=8, val=1, test=1


In [4]:
# Create dataset.yaml (task=obb, 3 classes by default)
# Adjust names if your classes differ.
DATA_YAML = ROOT / "dataset.yaml"
data_cfg = {
    "path": str(ROOT),
    "train": "train/images",
    "val":   "val/images",
    "test":  "test/images",
    "nc": 3,
    "names": {0: "card", 1: "face", 2: "text"},
    "task": "obb"
}
with open(DATA_YAML, "w") as f:
    yaml.safe_dump(data_cfg, f, sort_keys=False)

print("[OK] wrote", DATA_YAML)

[OK] wrote /Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/hand/dataset.yaml


In [5]:
# 1) Paths (EDIT ONLY IF NEEDED)
ROOT = Path("/Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/sorted_data/rot")
PREV_WEIGHTS = Path("/Users/shatha/Desktop/privacy-blurring-system/models/BG_v14/weights/best.pt")
PROJECT_DIR = Path("/Users/shatha/Desktop/privacy-blurring-system/models")
RUN_NAME = "BG_hand"
DATA_YAML = "/Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/hand/dataset.yaml" 

assert (ROOT/"train/images").exists() and (ROOT/"train/labels").exists(), "train split not found"
assert (ROOT/"val/images").exists() and (ROOT/"val/labels").exists(), "val split not found"
assert (ROOT/"test/images").exists() and (ROOT/"test/labels").exists(), "test split not found"
assert PREV_WEIGHTS.exists(), f"weights not found: {PREV_WEIGHTS}"
print("OK: dataset and weights exist.")

OK: dataset and weights exist.


In [6]:
# 2) Load model (fixed previous weights)
model = YOLO(str(PREV_WEIGHTS))
model.info()

YOLOv8s-obb summary: 144 layers, 11,422,940 parameters, 0 gradients, 29.6 GFLOPs


(144, 11422940, 0, 29.636684799999998)

In [7]:
# 3) Train
train_res = model.train(
    data=str(DATA_YAML),
    epochs=20,
    batch=8,
    imgsz=960,
    device="cpu",     
    workers=0,        
    amp=False,       
    patience=10,
    close_mosaic=10,
    seed=42,
    project=str(PROJECT_DIR),
    name=RUN_NAME,
    exist_ok=True,
    task="obb",
)
print("Train dir:", train_res.save_dir)

New https://pypi.org/project/ultralytics/8.3.197 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.196 🚀 Python-3.12.7 torch-2.8.0 CPU (Apple M2)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/hand/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/Users/shatha/Desktop/privacy-blurring-system/models/BG_v14/weights/best.p

/Users/shatha/Desktop/privacy-blurring-system/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3183.6±0.0 MB/s, size: 15935.6 KB)
val: Scanning /Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/hand/val/labels... 1 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1/1 223.6it/s 0.0s
val: New cache created: /Users/shatha/Desktop/privacy-blurring-system/aug_whitekey/hand/val/labels.cache
Plotting labels to /Users/shatha/Desktop/privacy-blurring-system/models/BG_hand/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
Image sizes 960 train, 960 val
Using 0 dataloader worke